# Sistema Integrado de Processamento de Imagens - Grupo 2

**Módulo Completo:** Análise de Métodos de Conversão, Análise de Filtros de Suavização, Teste de Ruído Sintético e Análise de Transformações de Contraste.

Este notebook implementa a suíte completa de análise de imagens para o Grupo 2.

# Célula 1: Importação das Bibliotecas

In [ ]:
import cv2 as cv
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, clear_output, HTML
import io
import zipfile
import os
from skimage.restoration import estimate_sigma
from skimage.util import random_noise
from skimage.metrics import structural_similarity as ssim

pd.set_option('display.float_format', lambda x: f'{x:.3f}')
plt.rcParams.update({'font.size': 10, 'figure.figsize': [10, 6]})

# Célula 2: Definição de Todas as Funções

In [ ]:
def converter_para_cinza(img_bgr):
    return cv.cvtColor(img_bgr, cv.COLOR_BGR2GRAY)

def plotar_histograma(img_cinza, ax):
    ax.set_title("Histograma de Tons de Cinza")
    ax.set_xlabel("Intensidade")
    ax.set_ylabel("Número de Pixels")
    ax.hist(img_cinza.ravel(), bins=256, range=[0, 256])
    ax.set_xlim([0, 256])
    ax.grid(True, linestyle='--', alpha=0.6)

def converter_para_cinza_varios_metodos(img_bgr):
    b, g, r = cv.split(img_bgr)
    img_cinza_padrao = cv.cvtColor(img_bgr, cv.COLOR_BGR2GRAY)
    img_cinza_media = np.mean(img_bgr, axis=2).astype(np.uint8)

    return {
        "Padrão": img_cinza_padrao,
        "Média": img_cinza_media,
        "Canal Vermelho": r,
        "Canal Verde": g,
        "Canal Azul": b
    }

def calcular_metricas_conversao(img_cinza):
    contraste_rms = np.std(img_cinza)
    imin, imax = np.min(img_cinza), np.max(img_cinza)
    contraste_michelson = 0.0 if (imax + imin) == 0 else (float(imax) - float(imin)) / (float(imax) + float(imin))

    hist = cv.calcHist([img_cinza], [0], None, [256], [0, 256])
    hist_norm = hist.ravel() / hist.sum()
    entropia = -np.sum(hist_norm * np.log2(hist_norm + 1e-6))

    return {
        "Contraste RMS": contraste_rms,
        "Contraste Michelson": contraste_michelson,
        "Entropia": entropia
    }

def plotar_conversoes_cinza(imagens_dict, nome_arquivo):
    num_imagens = len(imagens_dict)
    fig, axes = plt.subplots(1, num_imagens, figsize=(num_imagens * 4, 5))
    fig.suptitle(f"Comparação de Métodos de Conversão: {nome_arquivo}", fontsize=16)
    if num_imagens == 1: axes = [axes]
    for ax, (nome, img) in zip(axes, imagens_dict.items()):
        ax.imshow(img, cmap='gray', vmin=0, vmax=255)
        ax.set_title(nome)
        ax.axis('off')
    plt.tight_layout(rect=[0, 0.03, 1, 0.93])
    plt.show()

def medir_ruido_std(img):
    return np.std(img)

def medir_ruido_skimage(img_cinza):
    sigma = estimate_sigma(img_cinza, average_sigmas=True)
    return sigma if sigma > 1e-3 else 1e-3

def medir_foco_laplaciano(img_cinza):
    foco = cv.Laplacian(img_cinza, cv.CV_64F).var()
    return foco if foco > 1e-3 else 1e-3

def medir_ssim(img_original, img_filtrada):
    return ssim(img_original, img_filtrada, data_range=img_original.max() - img_original.min())

def aplicar_filtros(img_cinza):
    img_gaussiano = cv.GaussianBlur(img_cinza, (5, 5), 0)
    img_mediano = cv.medianBlur(img_cinza, 5)
    img_bilateral = cv.bilateralFilter(img_cinza, 9, 75, 75)
    return {
        'Gaussiano': img_gaussiano,
        'Mediano': img_mediano,
        'Bilateral': img_bilateral
    }

def avaliar_qualidade_filtros(df_resultados):
    if 'Original' not in df_resultados.index:
        return "Erro: 'Original' não encontrado nos resultados.", df_resultados

    ruido_orig = df_resultados.loc['Original', 'Ruído']
    foco_orig = df_resultados.loc['Original', 'Foco']
    ssim_orig = df_resultados.loc['Original', 'SSIM']

    df_filtros = df_resultados.drop('Original').copy()
    if df_filtros.empty:
        return "Nenhum filtro aplicado para avaliar.", df_resultados

    df_filtros['Redução Ruído'] = ((ruido_orig - df_filtros['Ruído']) / ruido_orig) * 100
    df_filtros['Perda Foco'] = ((foco_orig - df_filtros['Foco']) / foco_orig) * 100
    df_filtros['Perda SSIM'] = ((ssim_orig - df_filtros['SSIM']) / ssim_orig) * 100

    df_filtros['Score Final'] = (df_filtros['Redução Ruído'] * 0.5) - (df_filtros['Perda Foco'] * 0.3) - (df_filtros['Perda SSIM'] * 0.2)

    if df_filtros['Score Final'].empty:
        return "Não foi possível calcular o Score Final.", df_resultados

    melhor_filtro_nome = df_filtros['Score Final'].idxmax()
    melhor_filtro_stats = df_filtros.loc[melhor_filtro_nome]

    relatorio = f"O filtro {melhor_filtro_nome} foi o mais eficaz."

    df_final = pd.concat([df_resultados.loc[['Original']], df_filtros])
    df_final = df_final.fillna('-')
    return relatorio, df_final

def gerar_tabela_e_avaliar(img_cinza_original, imagens_filtradas):
    ruido_original = medir_ruido_skimage(img_cinza_original)
    foco_original = medir_foco_laplaciano(img_cinza_original)

    resultados = {
        'Original': {
            'Ruído': ruido_original,
            'Foco': foco_original,
            'SSIM': 1.0
        }
    }

    for nome_filtro, img_filtrada in imagens_filtradas.items():
        resultados[nome_filtro] = {
            'Ruído': medir_ruido_skimage(img_filtrada),
            'Foco': medir_foco_laplaciano(img_filtrada),
            'SSIM': medir_ssim(img_cinza_original, img_filtrada)
        }

    df_resultados = pd.DataFrame.from_dict(resultados, orient='index')
    relatorio_final, df_tabela = avaliar_qualidade_filtros(df_resultados)

    return relatorio_final, df_tabela

def adicionar_ruido_sintetico(imagem, tipo, valor):
    if tipo == 'Nenhum':
        return imagem

    imagem_float = imagem.astype(np.float32) / 255.0

    if tipo == 'Gaussiano':
        imagem_ruidosa = random_noise(imagem_float, mode='gaussian', var=valor)
    elif tipo == 'Sal e Pimenta':
        imagem_ruidosa = random_noise(imagem_float, mode='s&p', amount=valor)
    else:
        return imagem

    return np.uint8(np.clip(imagem_ruidosa * 255, 0, 255))


def aplicar_transformacao(img_cinza, metodo):
    if metodo == 'Nenhum':
        return img_cinza

    img_float = img_cinza.astype(np.float64)

    if metodo == 'Logaritmo':
        denominador = np.log(1 + np.max(img_float))
        if denominador == 0: return img_cinza
        c = 255 / denominador
        img_transformada = c * np.log(1 + img_float)
    elif metodo == 'Quadrado':
        img_norm = img_float / 255.0
        img_transformada = np.power(img_norm, 2.0) * 255
    elif metodo == 'Exponencial':
        img_norm = img_float / 255.0
        img_transformada = np.power(img_norm, 0.5) * 255
    elif metodo == 'Equalização de Histograma':
        img_transformada = cv.equalizeHist(img_cinza)
    else:
        return img_cinza

    return np.uint8(img_transformada)

def inverter_cores(img_cinza):
    return cv.bitwise_not(img_cinza)

imagens_carregadas = {}
imagem_selecionada_bgr = None
nome_arquivo_selecionado = ""
imagem_processada_final = None
download_lock = False

def medir_brilho_medio(img):
    return np.mean(img)

# Célula 3: Upload de Múltiplas Imagens

In [ ]:
imagens_carregadas = {}
imagem_selecionada_bgr = None
nome_arquivo_selecionado = ""

uploaded = files.upload()

if uploaded:
    arquivos_validos = 0
    arquivos_rejeitados = []

    for nome_arquivo, dados_arquivo in uploaded.items():
        extensao = nome_arquivo.split('.')[-1].lower()

        if extensao in ['jpg', 'jpeg', 'png']:
            try:
                buffer_imagem = np.frombuffer(dados_arquivo, np.uint8)
                imagem_bgr = cv.imdecode(buffer_imagem, cv.IMREAD_COLOR)
                if imagem_bgr is None: raise Exception("Não foi possível decodificar")

                imagens_carregadas[nome_arquivo] = imagem_bgr
                arquivos_validos += 1

            except Exception as e:
                pass
        else:
            arquivos_rejeitados.append(nome_arquivo)

    if arquivos_rejeitados:
        print("Os seguintes arquivos foram ignorados (extensão inválida):")
        for nome in arquivos_rejeitados:
            print(f"  - {nome}")
        print("\nPor favor, envie apenas arquivos .jpg, .jpeg ou .png.")

    if arquivos_validos == 0:
        pass
else:
    pass

# Célula 4: Visualização Inicial de Todas as Imagens

In [ ]:
if imagens_carregadas:
    for nome_arquivo, imagem_bgr in imagens_carregadas.items():
        imagem_rgb = cv.cvtColor(imagem_bgr, cv.COLOR_BGR2RGB)
        imagem_cinza_padrao = converter_para_cinza(imagem_bgr)

        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        fig.suptitle(f"{nome_arquivo} (Dimensões: {imagem_bgr.shape[1]}x{imagem_bgr.shape[0]})", y=1.02)

        axes[0].imshow(imagem_rgb)
        axes[0].set_title(f'Original (Colorida)')
        axes[0].axis('off')

        axes[1].imshow(imagem_cinza_padrao, cmap='gray')
        axes[1].set_title('Convertida (Padrão)')
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

else:
    pass

# Célula 5: Seleção da Imagem Ativa

In [ ]:
if imagens_carregadas:
    seletor_imagem = widgets.Dropdown(
        options=list(imagens_carregadas.keys()),
        description='Imagem Ativa:',
        disabled=False,
        layout=widgets.Layout(width='50%')
    )

    output_selecao = widgets.Output()

    def ao_selecionar_imagem(change):
        global imagem_selecionada_bgr, nome_arquivo_selecionado

        nome_arquivo_selecionado = change['new']
        imagem_selecionada_bgr = imagens_carregadas[nome_arquivo_selecionado]

        with output_selecao:
            clear_output(wait=True)
            imagem_rgb = cv.cvtColor(imagem_selecionada_bgr, cv.COLOR_BGR2RGB)

            plt.figure(figsize=(7, 5))
            plt.imshow(imagem_rgb)
            plt.title(f"Imagem Ativa: {nome_arquivo_selecionado}")
            plt.axis('off')
            plt.show()

    seletor_imagem.observe(ao_selecionar_imagem, names='value')

    display(seletor_imagem, output_selecao)

    if seletor_imagem.value:
        ao_selecionar_imagem({'new': seletor_imagem.value})

else:
    pass

# Célula 6: MÓDULO 0 - Análise de Métodos de Conversão

In [ ]:
if imagem_selecionada_bgr is not None:
    imagens_convertidas = converter_para_cinza_varios_metodos(imagem_selecionada_bgr)
    plotar_conversoes_cinza(imagens_convertidas, nome_arquivo_selecionado)

    resultados_metricas = {}
    for nome, img_cinza in imagens_convertidas.items():
        metricas = calcular_metricas_conversao(img_cinza)
        resultados_metricas[nome] = metricas

    df_conversao = pd.DataFrame.from_dict(resultados_metricas, orient='index')

    display(HTML(df_conversao.style.format('{:.4f}').to_html()))

    melhor_contraste = df_conversao['Contraste RMS'].idxmax()
    melhor_entropia = df_conversao['Entropia'].idxmax()

else:
    pass

# Célula 7: MÓDULO 1 - Análise de Filtros de Suavização (Pré-Processamento)

In [ ]:
if imagem_selecionada_bgr is not None:
    img_cinza_original = converter_para_cinza(imagem_selecionada_bgr)

    imagens_filtradas = aplicar_filtros(img_cinza_original)

    relatorio_final, df_tabela = gerar_tabela_e_avaliar(img_cinza_original, imagens_filtradas)

    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle(f"Comparação de Filtros: {nome_arquivo_selecionado}", fontsize=16)

    axes[0, 0].imshow(img_cinza_original, cmap='gray')
    axes[0, 0].set_title(f"Original\nRuído: {df_tabela.loc['Original', 'Ruído']:.4f} | Foco: {df_tabela.loc['Original', 'Foco']:.2f} | SSIM: 1.00")
    axes[0, 0].axis('off')

    idx_map = [(0, 1), (1, 0), (1, 1)]
    for i, (nome_filtro, img_filtrada) in enumerate(imagens_filtradas.items()):
        if nome_filtro in df_tabela.index:
            ax = axes[idx_map[i]]
            stats = df_tabela.loc[nome_filtro]
            ax.imshow(img_filtrada, cmap='gray')
            ax.set_title(f"Filtro {nome_filtro}\nRuído: {stats['Ruído']:.4f} | Foco: {stats['Foco']:.2f} | SSIM: {stats['SSIM']:.2f}")
            ax.axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95], h_pad=6)
    plt.show()

    def format_percent(v):
        return f'{v:.2f}%' if isinstance(v, (int, float)) else v

    def format_plus_percent(v):
        return f'{v:+.2f}%' if isinstance(v, (int, float)) else v

    def format_score_val(v):
        return f'{v:.2f}' if isinstance(v, (int, float)) else v

    format_dict = {
        'Ruído': '{:.4f}',
        'Foco': '{:.2f}',
        'SSIM': '{:.2f}',
        'Redução Ruído': format_percent,
        'Perda Foco': format_plus_percent,
        'Perda SSIM': format_plus_percent,
        'Score Final': format_score_val
    }

    def color_score(val):
        if isinstance(val, (int, float)):
            color = 'green' if val > 0 else 'red' if val < 0 else 'black'
            return f'color: {color}'
        return None

    styled_table = df_tabela.style.format(format_dict).map(color_score, subset=['Score Final', 'Redução Ruído', 'Perda Foco', 'Perda SSIM'])
    display(HTML(styled_table.to_html()))

    print(f"\n{relatorio_final}\n")

else:
    pass

# Célula 8: MÓDULO 2 - Teste de Estresse (Ruído Sintético)

In [ ]:
if imagem_selecionada_bgr is not None:

    tipo_ruido_dropdown = widgets.Dropdown(
        options=['Nenhum', 'Gaussiano', 'Sal e Pimenta'],
        value='Nenhum',
        description='Tipo de Ruído:',
        layout=widgets.Layout(width='50%')
    )

    valor_ruido_slider = widgets.FloatSlider(
        value=0.01,
        min=0.0,
        max=0.1,
        step=0.005,
        description='Intensidade:',
        readout_format='.3f',
        layout=widgets.Layout(width='50%')
    )

    output_area_mod2 = widgets.Output()

    def on_noise_controls_change(change):
        with output_area_mod2:
            clear_output(wait=True)

            tipo_ruido = tipo_ruido_dropdown.value
            valor_ruido = valor_ruido_slider.value

            img_cinza_base = converter_para_cinza(imagem_selecionada_bgr)

            img_com_ruido = adicionar_ruido_sintetico(img_cinza_base, tipo_ruido, valor_ruido)

            imagens_filtradas = aplicar_filtros(img_com_ruido)

            relatorio_final, df_tabela = gerar_tabela_e_avaliar(img_cinza_base, imagens_filtradas)
            df_tabela.loc['Original (Com Ruído)'] = [
                medir_ruido_skimage(img_com_ruido),
                medir_foco_laplaciano(img_com_ruido),
                medir_ssim(img_cinza_base, img_com_ruido),
                '-', '-', '-', '-'
            ]
            df_tabela = df_tabela.reindex(['Original', 'Original (Com Ruído)', 'Gaussiano', 'Mediano', 'Bilateral'])

            fig, axes = plt.subplots(2, 2, figsize=(12, 10))
            fig.suptitle(f"Teste de Estresse com Ruído {tipo_ruido}: {nome_arquivo_selecionado}", fontsize=16)

            axes[0, 0].imshow(img_com_ruido, cmap='gray')
            axes[0, 0].set_title(f"Original (Com Ruído {tipo_ruido})\nRuído: {df_tabela.loc['Original (Com Ruído)', 'Ruído']:.4f} | Foco: {df_tabela.loc['Original (Com Ruído)', 'Foco']:.2f} | SSIM: {df_tabela.loc['Original (Com Ruído)', 'SSIM']:.2f}")
            axes[0, 0].axis('off')

            idx_map = [(0, 1), (1, 0), (1, 1)]
            for i, (nome_filtro, img_filtrada) in enumerate(imagens_filtradas.items()):
                if nome_filtro in df_tabela.index:
                    ax = axes[idx_map[i]]
                    stats = df_tabela.loc[nome_filtro]
                    ax.imshow(img_filtrada, cmap='gray')
                    ax.set_title(f"Filtro {nome_filtro}\nRuído: {stats['Ruído']:.4f} | Foco: {stats['Foco']:.2f} | SSIM: {stats['SSIM']:.2f}")
                    ax.axis('off')

            plt.tight_layout(rect=[0, 0.03, 1, 0.95], h_pad=6)
            plt.show()

            def format_percent(v):
                return f'{v:.2f}%' if isinstance(v, (int, float)) else v

            def format_plus_percent(v):
                return f'{v:+.2f}%' if isinstance(v, (int, float)) else v

            def format_score_val(v):
                return f'{v:.2f}' if isinstance(v, (int, float)) else v

            def format_f_val(v, precision=4):
                 return f'{v:.{precision}f}' if isinstance(v, (int, float)) else v

            format_dict = {
                'Ruído': lambda v: format_f_val(v, 4),
                'Foco': lambda v: format_f_val(v, 2),
                'SSIM': lambda v: format_f_val(v, 2),
                'Redução Ruído': format_percent,
                'Perda Foco': format_plus_percent,
                'Perda SSIM': format_plus_percent,
                'Score Final': format_score_val
            }

            def color_score(val):
                if isinstance(val, (int, float)):
                    color = 'green' if val > 0 else 'red' if val < 0 else 'black'
                    return f'color: {color}'
                return None

            styled_table = df_tabela.style.format(format_dict).map(color_score, subset=['Score Final', 'Redução Ruído', 'Perda Foco', 'Perda SSIM'])

            display(HTML(styled_table.to_html()))

            print(f"\n{relatorio_final}\n")

    tipo_ruido_dropdown.observe(on_noise_controls_change, names='value')
    valor_ruido_slider.observe(on_noise_controls_change, names='value')

    display(widgets.VBox([
        widgets.HBox([tipo_ruido_dropdown, valor_ruido_slider]),
        output_area_mod2
    ]))

    on_noise_controls_change(None)

else:
    pass

# Célula 9: MÓDULO 3 - Transformações de Contraste e Manipulações (Pós-Processamento)

In [ ]:
if imagem_selecionada_bgr is not None:

    global algoritmo_dropdown, inverter_checkbox

    algoritmo_dropdown = widgets.Dropdown(
        options=['Nenhum', 'Logaritmo', 'Quadrado', 'Exponencial', 'Equalização de Histograma'],
        value='Nenhum',
        description='Transformação:',
        disabled=False,
        layout=widgets.Layout(width='50%')
    )

    inverter_checkbox = widgets.Checkbox(
        value=False,
        description='Inverter Cores (Negativo)',
        disabled=False
    )

    output_area_mod3 = widgets.Output()

    def on_controls_change_mod3(change):
        global imagem_processada_final

        with output_area_mod3:
            clear_output(wait=True)

            img_cinza = converter_para_cinza(imagem_selecionada_bgr)

            metodo = algoritmo_dropdown.value
            img_transformada = aplicar_transformacao(img_cinza, metodo)

            if inverter_checkbox.value:
                img_transformada = inverter_cores(img_transformada)

            imagem_processada_final = img_transformada

            fig, axes = plt.subplots(2, 2, figsize=(14, 10))
            fig.suptitle(f"Análise de Transformação: {metodo}", fontsize=16)

            axes[0, 0].imshow(img_cinza, cmap='gray')
            axes[0, 0].set_title("Imagem Original (Cinza)")
            axes[0, 0].axis('off')

            plotar_histograma(img_cinza, axes[0, 1])

            axes[1, 0].imshow(img_transformada, cmap='gray')
            axes[1, 0].set_title(f"Imagem Processada ({metodo})")
            axes[1, 0].axis('off')

            plotar_histograma(img_transformada, axes[1, 1])

            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            plt.show()

            brilho_antes = medir_brilho_medio(img_cinza)
            contraste_antes = medir_ruido_std(img_cinza)
            brilho_depois = medir_brilho_medio(img_transformada)
            contraste_depois = medir_ruido_std(img_transformada)

            df_metricas = pd.DataFrame({
                'Métrica': ['Brilho Médio', 'Contraste'],
                'Original (Cinza)': [brilho_antes, contraste_antes],
                'Processada': [brilho_depois, contraste_depois]
            })
            df_metricas['Mudança (%)'] = ((df_metricas['Processada'] - df_metricas['Original (Cinza)']) / (df_metricas['Original (Cinza)'] + 1e-6)) * 100

            df_metricas_display = df_metricas.set_index('Métrica')

            display(HTML(df_metricas_display.style.format({
                'Original (Cinza)': '{:.2f}',
                'Processada': '{:.2f}',
                'Mudança (%)': '{:+.2f}%'
            }).to_html()))

    algoritmo_dropdown.observe(on_controls_change_mod3, names='value')
    inverter_checkbox.observe(on_controls_change_mod3, names='value')

    display(widgets.VBox([
        widgets.HBox([algoritmo_dropdown, inverter_checkbox]),
        output_area_mod3
    ]))

    on_controls_change_mod3(None)

else:
    pass

# Célula 10: MÓDULO 4 - Download em Lote (ZIP)

In [ ]:
if 'nome_arquivo_selecionado' in globals() and nome_arquivo_selecionado:
    DOWNLOAD_EM_ANDAMENTO = False
    nome_arquivo_zip = f"Processamento_{nome_arquivo_selecionado.split('.')[0]}.zip"

    imagens_disponiveis = {}
    checkboxes = []


    botao_selecionar = widgets.Button(
        description='Baixar Imagens Selecionadas',
        button_style='info',
        tooltip='Selecionar imagens específicas para download',
        icon='check-square'
    )

    botao_baixar_todas = widgets.Button(
        description=f'Baixar TODAS as Imagens',
        button_style='danger',
        tooltip='Baixar todas as versões processadas da imagem',
        icon='download'
    )

    botao_baixar_selecionadas = widgets.Button(
        description='Baixar',
        button_style='success',
        tooltip='Baixar as imagens selecionadas',
        icon='download',
        disabled=True
    )

    output_download = widgets.Output()
    output_selecao = widgets.Output()

    def salvar_imagem_no_zip(img_array, sufixo, zip_file):
        from PIL import Image
        import io

        nome_base = nome_arquivo_selecionado.split('.')[0]
        nome_imagem = f"{nome_base}{sufixo}.png"

        img_pil = Image.fromarray(img_array.astype('uint8'))

        buffer = io.BytesIO()
        img_pil.save(buffer, format="PNG")
        buffer.seek(0)

        zip_file.writestr(nome_imagem, buffer.read())

    def processar_e_armazenar_imagens():
        global imagens_disponiveis
        imagens_disponiveis = {}

        if nome_arquivo_selecionado in imagens_carregadas:
             img_original = imagens_carregadas[nome_arquivo_selecionado]
        else:
            try:
                img_original = cv.imread(f'./{nome_arquivo_selecionado}')
                if img_original is None:
                     raise FileNotFoundError(f"Imagem '{nome_arquivo_selecionado}' não encontrada no disco.")
            except Exception as e:
                 print(f"Erro ao carregar a imagem original: {e}. Verifique se a Célula 3 foi executada.")
                 return

        if img_original is None:
            raise FileNotFoundError(f"Imagem '{nome_arquivo_selecionado}' não foi carregada.")

        imagens_disponiveis['Original (Colorida)'] = {
            'array': cv.cvtColor(img_original, cv.COLOR_BGR2RGB),
            'sufixo': '_original_colorida'
        }

        img_cinza_padrao = cv.cvtColor(img_original, cv.COLOR_BGR2GRAY)
        imagens_disponiveis['Cinza (Padrão)'] = {
            'array': img_cinza_padrao,
            'sufixo': '_cinza_padrao'
        }

        imagens_filtradas_padrao = aplicar_filtros(img_cinza_padrao)
        for nome_filtro, img_filtrada_padrao in imagens_filtradas_padrao.items():
            sufixo_filtro = f"_filtro_{nome_filtro.lower()}"
            imagens_disponiveis[f'Filtro {nome_filtro}'] = {
                'array': img_filtrada_padrao,
                'sufixo': sufixo_filtro
            }

        tipos_ruido = ['Gaussiano', 'Sal e Pimenta']
        valores_ruido = [0.05, 0.1]

        for tipo_ruido in tipos_ruido:
            for valor_ruido in valores_ruido:
                img_com_ruido = adicionar_ruido_sintetico(img_cinza_padrao, tipo_ruido, valor_ruido)
                sufixo_ruido = f"_ruido_{tipo_ruido.lower().replace(' ', '_')}_v{str(valor_ruido).replace('.', '')}"
                imagens_disponiveis[f'Ruído {tipo_ruido} ({valor_ruido})'] = {
                    'array': img_com_ruido,
                    'sufixo': sufixo_ruido
                }

                imagens_filtradas_ruido = aplicar_filtros(img_com_ruido)
                for nome_filtro, img_filtrada_ruido in imagens_filtradas_ruido.items():
                    sufixo_filtro_ruido = f"{sufixo_ruido}_filtro_{nome_filtro.lower()}"
                    imagens_disponiveis[f'Ruído {tipo_ruido} ({valor_ruido}) + Filtro {nome_filtro}'] = {
                        'array': img_filtrada_ruido,
                        'sufixo': sufixo_filtro_ruido
                    }

        metodos_transformacao = ['Logaritmo', 'Quadrado', 'Exponencial', 'Equalização de Histograma']
        for metodo in metodos_transformacao:
            img_transformada = aplicar_transformacao(img_cinza_padrao, metodo)
            sufixo = f"_transformacao_{metodo.lower().replace(' ', '_')}"
            imagens_disponiveis[f'Transformação {metodo}'] = {
                'array': img_transformada,
                'sufixo': sufixo
            }

            img_invertida = inverter_cores(img_transformada)
            sufixo_invertido = f"{sufixo}_invertido"
            imagens_disponiveis[f'Transformação {metodo} (Invertido)'] = {
                'array': img_invertida,
                'sufixo': sufixo_invertido
            }


    def baixar_todas_imagens(b):
        global DOWNLOAD_EM_ANDAMENTO
        if DOWNLOAD_EM_ANDAMENTO:
            print("Um download já está em andamento. Aguarde.")
            return

        DOWNLOAD_EM_ANDAMENTO = True
        botao_baixar_todas.disabled = True

        with output_download:
            clear_output(wait=True)
            print(f"Iniciando processamento e compressão para '{nome_arquivo_selecionado}'...")

        try:
            processar_e_armazenar_imagens()

            with zipfile.ZipFile(nome_arquivo_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
                for nome_img, dados in imagens_disponiveis.items():
                    salvar_imagem_no_zip(dados['array'], dados['sufixo'], zf)

            print("\nCompressão concluída. Iniciando download...")
            files.download(nome_arquivo_zip)
            print("Download iniciado. Verifique a barra lateral do Colab.")

        except Exception as e:
            print(f"Ocorreu um erro durante o download: {e}")
        finally:
            DOWNLOAD_EM_ANDAMENTO = False
            botao_baixar_todas.disabled = False

    def baixar_imagens_selecionadas(b):
        """Callback para o botão 'Baixar' (das selecionadas)."""
        global DOWNLOAD_EM_ANDAMENTO
        if DOWNLOAD_EM_ANDAMENTO:
            print("Um download já está em andamento. Aguarde.")
            return

        DOWNLOAD_EM_ANDAMENTO = True
        botao_baixar_selecionadas.disabled = True

        with output_selecao:
            clear_output(wait=True)
            print("Processando imagens selecionadas...")

        try:
            imagens_selecionadas = []
            for checkbox in checkboxes:
                if checkbox.value:
                    imagens_selecionadas.append(checkbox.description)

            if not imagens_selecionadas:
                with output_selecao:
                    print("Nenhuma imagem foi selecionada. Por favor, selecione ao menos uma imagem.")
                DOWNLOAD_EM_ANDAMENTO = False
                botao_baixar_selecionadas.disabled = False
                return

            nome_zip_selecionado = f"Selecionadas_{nome_arquivo_selecionado.split('.')[0]}.zip"
            with zipfile.ZipFile(nome_zip_selecionado, 'w', zipfile.ZIP_DEFLATED) as zf:
                for nome_img in imagens_selecionadas:
                    if nome_img in imagens_disponiveis:
                        dados = imagens_disponiveis[nome_img]
                        salvar_imagem_no_zip(dados['array'], dados['sufixo'], zf)

            with output_selecao:
                print(f"\n{len(imagens_selecionadas)} imagem(ns) selecionada(s). Iniciando download...")
            files.download(nome_zip_selecionado)
            with output_selecao:
                print("Download iniciado. Verifique a barra lateral do Colab.")

        except Exception as e:
            with output_selecao:
                print(f"Ocorreu um erro durante o download: {e}")
        finally:
            DOWNLOAD_EM_ANDAMENTO = False
            botao_baixar_selecionadas.disabled = False

    def mostrar_selecao_imagens(b):
        global checkboxes
        with output_selecao:
            clear_output(wait=True)
            print("Processando todas as imagens para seleção...")

        try:
            processar_e_armazenar_imagens()

            with output_selecao:
                clear_output(wait=True)
                print("Selecione as imagens que deseja baixar:\n")

            checkboxes = []
            for nome_img in imagens_disponiveis.keys():
                checkbox = widgets.Checkbox(
                    value=False,
                    description=nome_img,
                    disabled=False,
                    indent=False
                )
                checkboxes.append(checkbox)

            botao_baixar_selecionadas.disabled = False

            with output_selecao:
                display(widgets.VBox(checkboxes + [botao_baixar_selecionadas]))

        except Exception as e:
            with output_selecao:
                clear_output(wait=True)
                print(f"Ocorreu um erro ao processar as imagens: {e}")

    botao_selecionar.on_click(mostrar_selecao_imagens)
    botao_baixar_todas.on_click(baixar_todas_imagens)
    botao_baixar_selecionadas.on_click(baixar_imagens_selecionadas)

    display(HTML(f"<h3>Download de Imagens Processadas - {nome_arquivo_selecionado}</h3>"))
    display(HTML("<p>Escolha uma das opções abaixo:</p>"))
    display(widgets.HBox([botao_selecionar, botao_baixar_todas]))
    display(output_selecao)
    display(output_download)

else:
    display(HTML("<b>Nenhuma imagem carregada ou selecionada. Execute as células anteriores.</b>"))